**Caso de estudio - Elementos axisimétricos tipo lineal (triangulares)**

In [1]:
# Importación de módulos de cálculos
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'float': '{: 0.4f}'.format}) #Opciones de impresión de numpy

**Pre-procesamiento**

In [2]:
# Datos del problema
E = 200 * 10**9           #Módulo de elasticidad del acero [N/m^2]
v = 0.3                   #Módulo de Poisson del acero 
rho = 7850                #Densidad del acero [kg/m^3]
Sigma_adm = 200 * 10**6   #Esfuerzo de fluencia del acero [N/m^2]
a = 20 * 10**-3           #Longitud característica [m]
R = 6 * a                 #Radio externo del disco [m]

In [3]:
# Tabla de conectividades
# Elemento / nodo 1 / nodo 2 / nodo 3

Con = np.array([
        [0, 0, 1, 4],
        [1, 1, 2, 4],
        [2, 2, 5, 4],
        [3, 2, 6, 5],
        [4, 2, 3, 6],
        [5, 4, 5, 7],
        [6, 5, 6, 7],
        [7, 6, 8, 7]
    ])

# Tabla de coordenadas nodales
# Nodo / r / z

Coord = np.array([
        [0, 0, 0],
        [1, 0, a],
        [2, 0, 2*a],
        [3, 0, 3*a],
        [4, 3*a, a],
        [5, 3*a, 2*a],
        [6, 3*a, 3*a],
        [7, 6*a, 2*a],
        [8, 6*a, 3*a]
    ])

# Tabla de grados de libertad

GDL = np.array([
        [0, 1, 2, 3, 8, 9],       #Elemento 0
        [2, 3, 4, 5, 8, 9],       #Elemento 1
        [4, 5, 10, 11, 8, 9],     #Elemento 2
        [4, 5, 12, 13, 10, 11],   #Elemento 3
        [4, 5, 6, 7, 12, 13],     #Elemento 4
        [8, 9, 10, 11, 14, 15],   #Elemento 5
        [10, 11, 12, 13, 14, 15], #Elemento 6
        [12, 13, 16, 17, 14, 15]  #Elemento 7
    ])

nnodos = 9
ngdl_nodo = 2
nDOF_total = nnodos * ngdl_nodo

In [4]:
# Matriz del material

D = (E*(1-v))/((1+v)*(1-2*v)) * np.array([
                                    [      1, v/(1-v),                0, v/(1-v)],
                                    [v/(1-v),       1,                0, v/(1-v)],
                                    [      0,       0, (1-2*v)/(2*(1-v)),       0],
                                    [v/(1-v), v/(1-v),                0,        1]
                                 ])

In [5]:
# Función de cálculo de los parámetros de la matriz de derivadas de funciones de forma

def J(Coord, Con, elemento):
    r1 = Coord[Con[elemento, 1], 1]
    r2 = Coord[Con[elemento, 2], 1]
    r3 = Coord[Con[elemento, 3], 1]
    z1 = Coord[Con[elemento, 1], 2]
    z2 = Coord[Con[elemento, 2], 2]
    z3 = Coord[Con[elemento, 3], 2]
    r32 = r3 - r2
    r13 = r1 - r3
    r21 = r2 - r1
    r31 = - r13
    z23 = z2 - z3
    z31 = z3 - z1
    z12 = z1 - z2
    z21 = - z12
    detJ = r21*z31 - r31*z21
    
    return detJ, r1, r2, r3, r32, r13, r21, z23, z31, z12

In [6]:
# Función para el cálculo de la matriz de derivadas de funciones de forma simplificada

def Bb(Coord, Con, elemento):
    detJ, r1, r2, r3, r32, r13, r21, z23, z31, z12 = J(Coord, Con, elemento)
    Ae = detJ/2
    N1, N2, N3 = 1/3, 1/3, 1/3
    rm = (1/3) * (r1 + r2 + r3)
    
    B = (1/detJ) * np.array([
            [z23, 0, z31, 0, z12, 0],
            [0, r32, 0, r13, 0, r21],
            [r32, z23, r13, z31, r21, z12],
            [N1*detJ/rm, 0, N2*detJ/rm, 0, N3*detJ/rm, 0]
        ])
    
    return rm, Ae, B

In [7]:
# Ensamblaje de la matriz de rigidez

K = np.zeros([nDOF_total, nDOF_total])

for nelem in range(0, 8):
    rm, Ae, B = Bb(Coord, Con, nelem)
    DB = np.dot(D, B)
    BtDB = np.dot(np.transpose(B), DB)
    ke = 2 * np.pi * rm * Ae * BtDB
    for a in range(0, 6):
        rw = GDL[nelem, a]
        for b in range(0, 6):
            cl = GDL[nelem, b]
            K[rw, cl] = K[rw, cl] + ke[a, b]

In [8]:
# Función de cálculo de la carga centrífuga

def f_centrifuga(rho, rm, omega):
    fr = rho * rm * omega**2
    return fr

In [9]:
# Función de cargas de cuerpo elementales

def fe_cuerpo(rm, Ae, fr):
    fe = (2*np.pi/3) * rm * Ae * np.array([fr, 0, fr, 0, fr, 0])
    return fe

In [10]:
# Ensamblaje del vector de cargas

def Ensamblaje_carga(Coord, Con, rho, omega):
    
    F = np.zeros([nDOF_total])
    
    for nelem in range(0, 8):
        rm, Ae, B = Bb(Coord, Con, nelem)
        fr = f_centrifuga(rho, rm, omega)
        fe = fe_cuerpo(rm, Ae, fr)
        for a in range(0, 6):
            rw = GDL[nelem, a]
            F[rw] = F[rw] + fe[a]
    
    return F

**Resolución y Post-procesamiento**

In [11]:
# Aplicación de condiciones de borde por penalización
kc = np.max(np.max(np.abs(K))) * 10**6
Km = np.array(K)
Km[0,0] += kc
Km[2,2] += kc
Km[4,4] += kc
Km[6,6] += kc

# Método iterativo

omega = 2124.71    #Velocidad angular [rad/s]

F = Ensamblaje_carga(Coord, Con, rho, omega)

# Resolución del sistema
Q = np.linalg.solve(Km, F)
print("Desplazamientos nodales [mm]:")
print(Q * 10**3)
print()

np.allclose(np.dot(Km, Q), F)    #Verificación de la resolución del sistema

# Cálculo de esfuerzos
for nelem in range(0, 8):
    rm, Ae, B = Bb(Coord, Con, nelem)
    DB = np.dot(D, B)
    qe = np.zeros([6])
    for a in range(0, 6):
        rw = GDL[nelem, a]
        qe[a] = Q[rw]
    DBq = np.dot(DB, qe)
    print("Esfuerzos [MPa] del elemento", nelem)
    print(DBq * 10**-6)
    print()
    
print("Velocidad angular numérica máxima [rad/s]:")
print("%0.4f" % omega)

Desplazamientos nodales [mm]:
[ 0.0000  0.0374 -0.0000  0.0337 -0.0000  0.0314 -0.0000  0.0185  0.0132
  0.0097  0.0279  0.0042  0.0428 -0.0023  0.0385 -0.0383  0.0512 -0.0432]

Esfuerzos [MPa] del elemento 0
[ 63.0742  0.0000 -30.7039  63.0742]

Esfuerzos [MPa] del elemento 1
[ 71.8467  20.4693 -30.7039  71.8467]

Esfuerzos [MPa] del elemento 2
[ 133.2346  19.6633  21.5585  114.4462]

Esfuerzos [MPa] del elemento 3
[ 155.2056  33.0368  22.5047  174.3094]

Esfuerzos [MPa] del elemento 4
[ 199.9997 -8.9062 -26.7185  199.9997]

Esfuerzos [MPa] del elemento 5
[ 54.1871 -14.9490  1.7668  78.1193]

Esfuerzos [MPa] del elemento 6
[ 61.9485 -15.7851  2.7131  104.8268]

Esfuerzos [MPa] del elemento 7
[ 60.2800  1.1946 -3.5839  106.7386]

Velocidad angular numérica máxima [rad/s]:
2124.7100


In [12]:
# Solución analítica para un disco plano

omega_teo = np.sqrt((8*Sigma_adm/(3+v))*(1/(rho*(R)**2)))
print("Velocidad angular teórica máxima [rad/s]:")
print("%0.4f" % omega_teo)

Velocidad angular teórica máxima [rad/s]:
2071.0327


In [13]:
# Comparación de solución numérica contra analítica
error = np.abs(omega-omega_teo)*100/omega_teo
print("Error numérico [%]:")
print("%0.2f" % error)

Error numérico [%]:
2.59
